# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

## What we mean by “risk”
* **Beta** – covariance of a stock’s daily returns with the market (S&P 500) divided by the market’s variance. A beta < 1 implies milder swings; > 1 means amplified volatility.
* **52‑week price change** – simple percentage price move over the most recent 252 trading days, serving as a quick momentum gauge.

The two statistics are computed from free Yahoo Finance price data via `yfinance`. Feel free to swap in a different data source.

In [2]:
# --- 1. Install & import ---------------------------------
# !pip install --quiet yfinance pandas numpy matplotlib
import yfinance as yf
import pandas as pd, numpy as np, matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
# --- 2. Define ticker sets -------------------------------------------------
low_tickers  = ['JNJ', 'PG', 'KO', 'BRK-B', 'BND']
high_tickers = ['TSLA', 'NVDA', 'PLTR', 'COIN', 'MRNA']
market_ticker = '^GSPC'  # S&P 500 index
all_tickers = low_tickers + high_tickers + [market_ticker]

In [4]:
# --- 3. Download daily adjusted closes ------------------------------------
data = yf.download(all_tickers, start='2023-01-01', progress=False, auto_adjust=False)['Adj Close']
data.tail()

Ticker,BND,BRK-B,COIN,JNJ,KO,MRNA,NVDA,PG,PLTR,TSLA,^GSPC
Date,,,,,,,,,,,
2025-06-18,72.830002,485.140015,295.290009,150.729996,69.209999,25.400000,145.479996,158.020004,139.960007,322.049988,5980.870117
2025-06-20,72.879997,484.850006,308.380005,149.789993,68.839996,25.900000,143.850006,159.080002,137.300003,322.160004,5967.839844
2025-06-23,73.059998,487.769989,307.589996,151.320007,69.739998,25.670000,144.169998,161.029999,139.919998,348.679993,6025.169922
2025-06-24,73.279999,493.480011,344.820007,152.190002,70.209999,26.799999,147.899994,160.360001,143.229996,340.470001,6092.180176
2025-06-25,73.300003,486.209991,355.369995,152.279999,69.629997,26.860001,154.309998,158.970001,142.899994,327.549988,6092.160156


In [5]:
# --- 4. Compute daily log returns -----------------------------------------
returns = np.log(data / data.shift(1)).dropna()
returns.head()

Ticker,BND,BRK-B,COIN,JNJ,KO,MRNA,NVDA,PG,PLTR,TSLA,^GSPC
Date,,,,,,,,,,,
2023-01-04,0.005661,0.014861,0.115134,0.010829,-0.000477,-0.009148,0.029867,0.004345,0.029300,0.049979,0.007511
2023-01-05,-0.001102,-0.005259,-0.117220,-0.007411,-0.011509,-0.021946,-0.033366,-0.012493,-0.040315,-0.029469,-0.011714
2023-01-06,0.010967,0.018335,-0.008085,0.008077,0.019109,0.038664,0.040797,0.023534,0.012579,0.024352,0.022584
2023-01-09,0.002723,-0.009965,0.140311,-0.026250,-0.012539,0.019765,0.050459,-0.012289,0.015504,0.057655,-0.000768
2023-01-10,-0.003951,0.002595,0.121868,-0.002395,-0.007696,0.030516,0.017821,-0.000987,0.030305,-0.007711,0.006954


In [6]:
# --- 5. Calculate betas ----------------------------------------------------
market_returns = returns[market_ticker]
betas = {}
for ticker in all_tickers:
    if ticker == market_ticker:
        continue
    cov = np.cov(returns[ticker], market_returns)[0][1]
    var = market_returns.var()
    betas[ticker] = cov / var
pd.Series(betas, name='Beta').to_frame()

,Beta
JNJ,0.108563
PG,0.206792
KO,0.168740
BRK-B,0.603343
BND,0.048337
TSLA,2.300428
NVDA,2.168243
PLTR,2.306468
COIN,2.549799
MRNA,1.223159


In [7]:
# --- 6. 52‑week price change ----------------------------------------------
price_change = (data.iloc[-1] - data.iloc[-252]) / data.iloc[-252] * 100
price_change = price_change.drop(market_ticker).rename('52wk_return_%')
price_change.to_frame()

,52wk_return_%
Ticker,
BND,4.884288
BRK-B,17.447698
COIN,67.382601
JNJ,5.446348
KO,12.031148
MRNA,-80.394160
NVDA,30.690665
PG,-3.320772
PLTR,491.473488


In [8]:
# --- 7. Combine metrics ----------------------------------------------------
metrics = pd.concat([pd.Series(betas, name='Beta'), price_change], axis=1)
metrics

,Beta,52wk_return_%
JNJ,0.108563,5.446348
PG,0.206792,-3.320772
KO,0.168740,12.031148
BRK-B,0.603343,17.447698
BND,0.048337,4.884288
TSLA,2.300428,79.400802
NVDA,2.168243,30.690665
PLTR,2.306468,491.473488
COIN,2.549799,67.382601
MRNA,1.223159,-80.394160


In [9]:
# --- 8. Helper to compute portfolio stats ---------------------------------
def portfolio_stats(tickers):
    w = pd.Series(1/len(tickers), index=tickers)  # equal weights
    beta = (metrics.loc[w.index, 'Beta'] * w).sum()
    ret  = (metrics.loc[w.index, '52wk_return_%'] * w).sum()
    return beta, ret

low_beta, low_ret   = portfolio_stats(low_tickers)
high_beta, high_ret = portfolio_stats(high_tickers)

summary = pd.DataFrame({
    'Portfolio': ['Low‑Risk', 'High‑Risk'],
    'Avg_Beta': [low_beta, high_beta],
    'Weighted_52wk_return_%': [low_ret, high_ret]
})
summary

,Portfolio,Avg_Beta,Weighted_52wk_return_%
0,Low‑Risk,0.227155,7.297742
1,High‑Risk,2.109619,117.710679


## Interpretation
* **Low‑risk portfolio** – target beta well below 1 (typically ~0.5) thanks to defensive sectors and a bond ETF sleeve. Historically these names cushion drawdowns while still delivering modest gains.
* **High‑risk portfolio** – beta > 2 provides leveraged exposure to growth themes (AI chips, EVs, crypto); returns are more explosive but drawdowns equally severe.
